<a href="https://colab.research.google.com/github/yunjeong-chang/My_Data_project/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EB%B0%98%EB%8F%84%EC%B2%B4%20%EB%B0%95%EB%A7%89%20%EB%91%90%EA%BB%98%20%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np

#학습 및 테스트 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_mdc01/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_mdc01/test.csv')

print("train shape is", train.shape) # (데이터 81만개, 박막 두께 정보 4개 + 226개의 반사율 스펙트럼)
print("test shape is", test.shape)

train shape is (810000, 230)
test shape is (10000, 227)


In [15]:
# 학습데이터 살펴보기
train.head() # 박막의 두께는 10단위의 정수형, 반사율 스펙트럼값은 소수점 단위

,layer_1,layer_2,layer_3,layer_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225
0,10,10,10,10,0.254551,0.258823,0.254659,0.252085,0.247678,0.253614,0.246511,0.259407,0.260862,0.242524,0.253870,0.245156,0.245548,0.255501,0.228948,0.228632,0.225802,0.249418,0.246910,0.248747,0.251088,0.244886,0.233906,0.242632,0.221792,0.236521,0.220555,0.243761,0.230202,0.226122,0.220671,0.235075,0.224560,0.226998,0.209499,0.226594,...,0.160126,0.164192,0.171406,0.162364,0.168591,0.166626,0.169696,0.196383,0.194145,0.190646,0.186797,0.191748,0.190886,0.208518,0.211220,0.229286,0.226265,0.226954,0.229023,0.237112,0.262421,0.262566,0.272062,0.292049,0.305353,0.292889,0.317479,0.316911,0.321371,0.355636,0.354750,0.369223,0.388184,0.408496,0.414564,0.429403,0.419225,0.443250,0.433414,0.465502
1,10,10,10,20,0.205062,0.225544,0.217758,0.202169,0.199633,0.207380,0.191318,0.195369,0.200536,0.197588,0.198726,0.191803,0.199625,0.206465,0.182836,0.193341,0.180459,0.196416,0.185398,0.188913,0.176011,0.173457,0.182249,0.176816,0.192716,0.185924,0.183816,0.186900,0.161040,0.173525,0.172948,0.182634,0.161153,0.158073,0.152335,0.148293,...,0.327076,0.331195,0.337722,0.340189,0.342748,0.351925,0.372989,0.389663,0.396216,0.384977,0.383701,0.403894,0.406382,0.429342,0.409908,0.439307,0.432569,0.439114,0.459541,0.454988,0.475268,0.478575,0.483466,0.496912,0.499542,0.522705,0.524767,0.533952,0.558729,0.572470,0.557203,0.573656,0.587998,0.612754,0.627825,0.633393,0.637706,0.625981,0.653231,0.637853
2,10,10,10,30,0.189196,0.165869,0.177655,0.156822,0.175094,0.177755,0.157582,0.158885,0.156911,0.166162,0.148831,0.144950,0.151362,0.145110,0.159201,0.139296,0.145313,0.156555,0.145363,0.150352,0.150542,0.142990,0.139572,0.145669,0.144514,0.126263,0.112832,0.117535,0.111473,0.117545,0.107058,0.119299,0.110768,0.124217,0.124496,0.110023,...,0.525450,0.532645,0.539097,0.527091,0.531925,0.530325,0.551716,0.556125,0.567470,0.549711,0.566789,0.574313,0.569873,0.574390,0.601672,0.584206,0.602981,0.598708,0.615189,0.637023,0.626458,0.645439,0.629203,0.640967,0.652762,0.660495,0.676498,0.674599,0.693535,0.699182,0.699864,0.708688,0.721982,0.713464,0.743030,0.741709,0.747743,0.746037,0.737356,0.750391
3,10,10,10,40,0.131003,0.120076,0.138975,0.117931,0.130566,0.131262,0.126962,0.134453,0.106717,0.127309,0.099958,0.112908,0.106853,0.108288,0.101393,0.094074,0.086854,0.099669,0.107276,0.091117,0.084525,0.078293,0.076476,0.078559,0.072960,0.075898,0.067963,0.066055,0.087904,0.065373,0.075469,0.080097,0.068895,0.058148,0.057882,0.063258,...,0.616934,0.638654,0.630914,0.647135,0.629796,0.644114,0.641825,0.645360,0.653030,0.656335,0.657734,0.678019,0.676713,0.681540,0.673333,0.686743,0.697500,0.702650,0.693750,0.711013,0.714411,0.701010,0.726972,0.719394,0.726329,0.730821,0.740062,0.747356,0.749405,0.766173,0.764786,0.763788,0.770017,0.787571,0.778866,0.776969,0.774712,0.801526,0.805305,0.784057
4,10,10,10,50,0.091033,0.086893,0.108125,0.080405,0.105917,0.077083,0.097895,0.086765,0.078676,0.075729,0.086023,0.070649,0.078957,0.072772,0.069867,0.080523,0.068273,0.074660,0.069852,0.047442,0.067855,0.049580,0.068737,0.041386,0.048697,0.040278,0.050234,0.059371,0.042581,0.037117,0.041503,0.055113,0.034001,0.035846,0.030385,0.048935,...,0.670888,0.692846,0.673025,0.704448,0.696657,0.697524,0.692513,0.714224,0.696880,0.716361,0.723607,0.720018,0.715039,0.705079,0.720913,0.723440,0.719858,0.729948,0.731360,0.728861,0.740501,0.738362,0.749341,0.738456,0.768791,0.770292,0.766630,0.779970,0.787695,0.766521,0.786677,0.802271,0.806557,0.799614,0.789333,0.804087,0.787763,0.794948,0.819105,0.801781


In [24]:
#기본적인 통계 살펴보기
train.loc[:,'0':'225'].describe() 
# 반사율 스펙트럼값 부분만 출력
# max값이 1을 넘지 않으므로 따로 정규화가 필요 없음을 확인!

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225
count,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,...,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000,810000.000000
mean,0.292653,0.292893,0.293125,0.293363,0.293666,0.293994,0.294344,0.294792,0.295325,0.295757,0.296188,0.296762,0.297396,0.298059,0.298745,0.299503,0.300220,0.301024,0.301834,0.302585,0.303395,0.304331,0.305255,0.306159,0.307080,0.307916,0.308737,0.309647,0.310584,0.311537,0.312403,0.313251,0.314086,0.314904,0.315711,0.316522,0.317295,0.318028,0.318752,0.319424,...,0.515799,0.516212,0.517059,0.518330,0.519609,0.520906,0.522183,0.523519,0.524838,0.526181,0.527548,0.528943,0.530954,0.533822,0.536751,0.539718,0.542706,0.545724,0.548767,0.551864,0.554950,0.558064,0.561231,0.565091,0.569481,0.574082,0.578907,0.583966,0.589218,0.594688,0.600336,0.606206,0.612238,0.618456,0.623942,0.625395,0.627100,0.628997,0.631166,0.633594
std,0.181642,0.181857,0.182055,0.182197,0.182361,0.182529,0.182693,0.182858,0.183057,0.183192,0.183337,0.183532,0.183731,0.183932,0.184135,0.184367,0.184559,0.184800,0.185021,0.185242,0.185466,0.185706,0.185972,0.186212,0.186465,0.186681,0.186877,0.187115,0.187331,0.187556,0.187732,0.187905,0.188082,0.188232,0.188361,0.188493,0.188607,0.188717,0.188786,0.188883,...,0.209915,0.209691,0.209475,0.209221,0.208968,0.208739,0.208539,0.208366,0.208186,0.208027,0.207935,0.207825,0.207651,0.207415,0.207167,0.206929,0.206699,0.206484,0.206228,0.205982,0.205727,0.205514,0.205216,0.204812,0.204335,0.203782,0.203131,0.202415,0.201604,0.200721,0.199727,0.198644,0.197473,0.196177,0.195028,0.194909,0.194730,0.194493,0.194146,0.193725
min,-0.014902,-0.014798,-0.014897,-0.014709,-0.014903,-0.014662,-0.014924,-0.014789,-0.014775,-0.014945,-0.014818,-0.014792,-0.014751,-0.014926,-0.014878,-0.014824,-0.014904,-0.014811,-0.014950,-0.014979,-0.014780,-0.014970,-0.014883,-0.015064,-0.014947,-0.014940,-0.014819,-0.014870,-0.014717,-0.014732,-0.014987,-0.014754,-0.014905,-0.014728,-0.014683,-0.014827,-0.015003,-0.014727,-0.014758,-0.014817,...,-0.014814,-0.015110,-0.014201,-0.014504,-0.014645,-0.014396,-0.014689,-0.014374,-0.014852,-0.014072,-0.014635,-0.014340,-0.014414,-0.014129,-0.013891,-0.014385,-0.014613,-0.013654,-0.014283,-0.014053,-0.014433,-0.013465,-0.010545,-0.014733,-0.014524,-0.014323,-0.013598,-0.012928,-0.012640,-0.012806,-0.011992,-0.008661,-0.011430,-0.009827,-0.007632,-0.007411,-0.007073,-0.007101,-0.005519,-0.006074
25%,0.135139,0.135180,0.135258,0.135478,0.135585,0.135705,0.135858,0.136423,0.136704,0.136993,0.137269,0.137756,0.138130,0.138556,0.139140,0.139715,0.140296,0.140825,0.141515,0.142154,0.142753,0.143431,0.144334,0.145081,0.145881,0.146606,0.147193,0.148069,0.148928,0.149679,0.150563,0.151352,0.152184,0.152947,0.153714,0.154655,0.

# **0. 데이터 수를 좀 줄일 순 없을까?**

"pandas의 sample함수"

n = 추출할 데이터의 개수

frac = 추출할 데이터 비율

replace = True(복원), False(비복원)

'실제 대회에선 당연히 데이터 수가 많아야 좋지만, 지금은 공부니까 시간이 너무 오래걸리면 안된다.. 

데이콘 코드공유에 보니까 PCA를 통한 학습시간 단축 효과에 관한 글이 있던데 이것도 해보면 좋을 것 같음'

In [6]:
train.sample(frac = 0.01) # 81만개에서 8100개로 줄여서 해보자

,layer_1,layer_2,layer_3,layer_4,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,...,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225
483695,180,280,140,60,0.436156,0.445514,0.467728,0.473762,0.455696,0.475281,0.459427,0.477176,0.445042,0.448936,0.421368,0.410206,0.390364,0.385105,0.362209,0.349018,0.320814,0.299091,0.247143,0.227472,0.193617,0.177749,0.156689,0.158788,0.131151,0.142471,0.124232,0.140518,0.156973,0.193099,0.216391,0.231050,0.285332,0.291189,0.344715,0.375304,...,0.201253,0.132766,0.065962,0.054640,0.069196,0.132167,0.190393,0.279089,0.378670,0.433587,0.494671,0.562966,0.621985,0.662701,0.679248,0.710397,0.736840,0.767403,0.779975,0.787654,0.781140,0.805831,0.809346,0.792602,0.789414,0.810985,0.786305,0.780756,0.788005,0.776328,0.747245,0.753928,0.716498,0.706248,0.669313,0.607518,0.569711,0.525835,0.479406,0.430053
195263,80,70,290,240,0.448806,0.452059,0.441054,0.419095,0.414087,0.377398,0.378499,0.348677,0.315418,0.268123,0.250076,0.195030,0.167321,0.139570,0.105737,0.081397,0.052411,0.067618,0.050084,0.082961,0.105953,0.145357,0.164304,0.222081,0.254978,0.302149,0.356120,0.384149,0.424361,0.452175,0.493941,0.523245,0.533102,0.545875,0.560767,0.576744,...,0.666295,0.654572,0.667583,0.664832,0.671773,0.665112,0.657439,0.637521,0.614333,0.589705,0.585240,0.539621,0.493831,0.444016,0.399445,0.353304,0.315266,0.274085,0.259313,0.278249,0.303872,0.334928,0.410005,0.488647,0.554945,0.611225,0.669560,0.707120,0.750691,0.777343,0.817524,0.811514,0.832283,0.856097,0.852927,0.866278,0.885705,0.869930,0.882202,0.878879
656444,250,100,120,150,0.543541,0.562039,0.576244,0.607078,0.621596,0.614061,0.621092,0.649081,0.640955,0.656027,0.655673,0.647974,0.647600,0.660561,0.644388,0.625058,0.641260,0.622033,0.596479,0.590159,0.561551,0.569082,0.545304,0.500374,0.486379,0.443703,0.421032,0.363410,0.339614,0.295173,0.259291,0.211035,0.153077,0.115085,0.078822,0.032487,...,0.612455,0.578223,0.551765,0.501691,0.470164,0.393091,0.351316,0.322735,0.278499,0.277553,0.292875,0.324810,0.345000,0.421844,0.466305,0.511653,0.572508,0.632096,0.648225,0.674366,0.721721,0.746286,0.744112,0.785564,0.790752,0.794289,0.794746,0.823648,0.817825,0.814751,0.821993,0.814357,0.804089,0.817605,0.806876,0.793214,0.793461,0.772191,0.750883,0.706906
642089,240,240,130,300,0.414941,0.449735,0.434684,0.439262,0.465074,0.444656,0.435848,0.419013,0.397554,0.385356,0.350777,0.326029,0.266858,0.217404,0.179747,0.120122,0.096543,0.038756,0.033646,0.030832,0.052779,0.084403,0.141090,0.188981,0.241713,0.312239,0.377156,0.430975,0.472920,0.495576,0.556450,0.565784,0.602246,0.598371,0.619206,0.629449,...,0.511502,0.503278,0.507731,0.505864,0.463410,0.440102,0.378217,0.319307,0.260276,0.188520,0.122003,0.048100,0.040999,0.050521,0.122123,0.239964,0.352678,0.440296,0.536399,0.601641,0.660904,0.722804,0.738024,0.764285,0.806248,0.810878,0.837460,0.821271,0.829290,0.840668,0.829897,0.850224,0.841396,0.819140,0.823871,0.796422,0.769164,0.737888,0.662409,0.611174
138868,60,50,90,290,0.264355,0.240068,0.234682,0.244630,0.229958,0.218465,0.203380,0.194537,0.162529,0.134942,0.128692,0.103677,0.091251,0.094139,0.060603,0.063544,0.025131,0.019306,0.030626,0.020562,0.025144,0.023093,0.039023,0.032228,0.052609,0.088713,0.101484,0.140851,0.177646,0.196912,0.231786,0.275208,0.300752,0.344516,0.369158,0.380829,...,0.743159,0.747303,0.749173,0.749730,0.716913,0.710689,0.700030,0.682861,0.672009,0.665114,0.643364,0.634882,0.596867,0.586800,0.541681,0.505325,0.479330,0.436903,0.412877,0.383031,0.329101,0.291199,0.256772,0.215560,0.184924,0.167149,0.181588,0.165284,0.194899,0.226396,0.258838,0.275639,0.334886,0.368832,0.400197,0.439718,0.464229,0.516832,0.515095,0.553079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

* 데이터 좋아서 따로 전처리 필요 없음, 결측치도 없음

In [26]:
train.isnull().sum() #결측치 찾는 함수 기억해두자

layer_1    0
layer_2    0
layer_3    0
layer_4    0
0          0
          ..
221        0
222        0
223        0
224        0
225        0
Length: 230, dtype: int64

In [16]:
#데이터 슬라이싱
train_x = train.iloc[:,4:]
train_y = train.iloc[:,0:4] #layer층 네 개 따로 분리한 것

test_x = test.iloc[:,1:] #ID 클래스 빼버린 것

# **1. 베이스라인 코드**

In [28]:
#케라스를 통해 모델 생성을 시작합니다.
import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=160, activation='relu', input_dim=226))
model.add(Dense(units=160, activation='relu'))
model.add(Dense(units=160, activation='relu'))
model.add(Dense(units=4, activation='linear'))

In [29]:
#모델을 컴파일합니다.
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [31]:
#모델을 학습합니다.
model.fit(train_x, train_y, epochs=20, batch_size=10000, validation_split = 0.05)

Epoch 1/20
77/77 [==============================] - 10s 124ms/step - loss: 121.2578 - mae: 121.2578 - val_loss: 95.8219 - val_mae: 95.8219
Epoch 2/20
77/77 [==============================] - 9s 119ms/step - loss: 74.7750 - mae: 74.7750 - val_loss: 93.4398 - val_mae: 93.4398
Epoch 3/20
77/77 [==============================] - 9s 119ms/step - loss: 73.1706 - mae: 73.1706 - val_loss: 93.0216 - val_mae: 93.0216
Epoch 4/20
77/77 [==============================] - 9s 119ms/step - loss: 69.9334 - mae: 69.9334 - val_loss: 91.7666 - val_mae: 91.7666
Epoch 5/20
77/77 [==============================] - 9s 119ms/step - loss: 65.2067 - mae: 65.2067 - val_loss: 89.5399 - val_mae: 89.5399
Epoch 6/20
77/77 [==============================] - 9s 119ms/step - loss: 62.9879 - mae: 62.9879 - val_loss: 86.8610 - val_mae: 86.8610
Epoch 7/20
77/77 [==============================] - 9s 119ms/step - loss: 62.1947 - mae: 62.1947 - val_loss: 84.6340 - val_mae: 84.6340
Epoch 8/20
77/77 [===========================

In [33]:
#예측값을 생성합니다.
pred_test = model.predict(test_x)

In [35]:
#submission 파일을 생성합니다.
sample_sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_mdc01/sample_submission.csv', index_col=0)
submission = sample_sub+pred_test
submission.to_csv('submission_baseline.csv')

- 이렇게만 제출하니까 52점대 나옴
- 다른 사람들 보니까 주어진 81만개 데이터로 돌리면 10점대 이내로 줄어든다고 함.

# **2. 코드 공유**
- 다른 분이 베이스라인 기반으로 층 추가, Dropout 추가하셨는데, 하이퍼파라미터 조금 더 수정해 봄

In [8]:
# 베이스 라인 모델을 기반으로 약간의 추가 진행
# Baseline 모델이 얇다고 생각해서 layer 추가 -> 또 너무 많으니까 시간 오래걸려서 적당히 조정
# train loss는 매우 빠르게 감소하나 val loss가 잘 감소하지 않음
# 과적합을 고려해서 Dropout 추가 0.5 넣으니 매우 성능 감소 -> 적당히 줄이고 조정

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.layers import Activation, Dropout, BatchNormalization, Input
from keras.layers import *

model = Sequential()
model.add(Dense(units=226, activation='relu', input_dim=226))
model.add(Dropout(0.05))
model.add(Dense(226, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(226, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(226, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(226, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(226, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(226, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(units=4, activation='relu'))

In [9]:
#모델을 컴파일합니다.
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [10]:
#모델을 학습합니다.
model.fit(train_x, train_y, epochs=20, batch_size=10000, validation_split = 0.05)

Epoch 1/20
77/77 [==============================] - 49s 623ms/step - loss: 110.0623 - mae: 110.0623 - val_loss: 93.2787 - val_mae: 93.2787
Epoch 2/20
77/77 [==============================] - 47s 613ms/step - loss: 71.3929 - mae: 71.3929 - val_loss: 84.2635 - val_mae: 84.2635
Epoch 3/20
77/77 [==============================] - 47s 616ms/step - loss: 64.1158 - mae: 64.1158 - val_loss: 79.9810 - val_mae: 79.9810
Epoch 4/20
77/77 [==============================] - 47s 617ms/step - loss: 62.7335 - mae: 62.7335 - val_loss: 79.0985 - val_mae: 79.0985
Epoch 5/20
77/77 [==============================] - 47s 614ms/step - loss: 61.9839 - mae: 61.9839 - val_loss: 78.0348 - val_mae: 78.0348
Epoch 6/20
77/77 [==============================] - 47s 614ms/step - loss: 61.6105 - mae: 61.6105 - val_loss: 76.4613 - val_mae: 76.4613
Epoch 7/20
77/77 [==============================] - 47s 616ms/step - loss: 60.1769 - mae: 60.1769 - val_loss: 70.1263 - val_mae: 70.1263
Epoch 8/20
77/77 [=====================

In [11]:
#예측값을 생성합니다.
pred_test = model.predict(test_x)

In [12]:
#submission 파일을 생성합니다.
sample_sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_mdc01/sample_submission.csv', index_col=0)
submission = sample_sub+pred_test
submission.to_csv('submission_baseline2.csv')

- 엥 시간 엄청 오래 걸리네
- 점수는 24점 정도로 확 줄어들긴 함

# **3. 전체 데이터 이용**
- 시간 좀 오래 걸려도 전체 데이터가지고 돌려보자
- 모델도 더 간단하게

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Activation, Dropout, BatchNormalization, Input
from keras.layers import *

model = Sequential()
model.add(Dense(3000,input_dim=226,activation='relu')) # 입력층

model.add(Dense(1000,activation='relu')) # 은닉층
model.add(BatchNormalization())
model.add(Dropout(0.05))

model.add(Dense(500,activation='relu')) # 은닉층
model.add(BatchNormalization())
model.add(Dropout(0.05))

model.add(Dense(300,activation='relu')) # 은닉층
model.add(BatchNormalization())

model.add(Dense(4))# 출력층 ,출력 4개

In [18]:
#모델을 컴파일합니다.
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [19]:
#모델을 학습합니다.
early_stop = EarlyStopping(monitor='val_loss',patience=5,verbose=1)
model.fit(train_x, train_y, epochs=20, batch_size=10000, validation_split = 0.05, callbacks=[early_stop]) 

Epoch 1/20
77/77 [==============================] - 366s 5s/step - loss: 152.3725 - mae: 152.3725 - val_loss: 189.5516 - val_mae: 189.5516
Epoch 2/20
77/77 [==============================] - 365s 5s/step - loss: 146.7797 - mae: 146.7797 - val_loss: 102.6954 - val_mae: 102.6954
Epoch 3/20
77/77 [==============================] - 364s 5s/step - loss: 131.7830 - mae: 131.7830 - val_loss: 105.6747 - val_mae: 105.6747
Epoch 4/20
77/77 [==============================] - 371s 5s/step - loss: 108.1253 - mae: 108.1253 - val_loss: 125.5375 - val_mae: 125.5375
Epoch 5/20
77/77 [==============================] - 364s 5s/step - loss: 83.1090 - mae: 83.1090 - val_loss: 91.5101 - val_mae: 91.5101
Epoch 6/20
77/77 [==============================] - 362s 5s/step - loss: 63.8508 - mae: 63.8508 - val_loss: 112.8150 - val_mae: 112.8150
Epoch 7/20
77/77 [==============================] - 363s 5s/step - loss: 45.3845 - mae: 45.3845 - val_loss: 102.4584 - val_mae: 102.4584
Epoch 8/20
77/77 [=================

In [20]:
#예측값을 생성합니다.
pred_test = model.predict(test_x) 

In [21]:
#submission 파일을 생성합니다.
sample_sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_mdc01/sample_submission.csv', index_col=0)
submission = sample_sub+pred_test
submission.to_csv('submission_baseline3.csv')

- 시간이 더더더더더더더 많이 걸린다.. 두시간 걸렸다.
- 점수는 9.2점 ! 
- 모델이 단순해도 데이터 많이 쓰니까 점수가 확 좋아지는걸 봐서는 데이터가 중요하긴 한가보다.. 

  여기서 모델 더 신경쓰면 점수 더 줄일 수 있을 듯!

- 파이토치나 앙상블 같은 부분 자세히 공부 못한 게 아쉽지만, 

  튜토리얼 캐글이나 실습용 데이터만 해보다가 나름 실전?스러운 문제를 다뤄봤다는 자체로 의미있는 시작이라고 생각한다!
- 다음 주제부턴 더 꼼꼼하게 열심히 공부해야지!!